In [1]:
import os
import pandas as pd
import numpy as np

# MBPP

In [2]:
output_dir = '../llm_output'
mbpp_dir = os.path.join(output_dir, 'mbpp')
mbpp_df = pd.read_csv(os.path.join(mbpp_dir, 'codellama-CodeLlama-7b-Instruct-hf_predictions.csv'))

In [3]:
len(mbpp_df)

289200

In [4]:
mbpp_df.columns

Index(['id', 'text', 'generated_text', 'seed', 'max_new_tokens', 'do_sample',
       'stop_sequences', 'temperature', 'top_p', 'response_time',
       'tokens_per_second', 'milliseconds_per_token', 'torch_allocated_GBs',
       'total_GPU_memory_used_GBs', 'GPU_memory_used_GBs', 'finish_reason',
       'generated_tokens', 'task_id', 'hypothesis', 'dataset',
       'model_name_or_path', 'generated_code', 'test_code', 'passed', 'result',
       'hash'],
      dtype='object')

In [5]:
mbpp_df["passed"].isna().sum()

0

In [6]:
mbpp_df.head()

,id,text,generated_text,seed,max_new_tokens,do_sample,stop_sequences,temperature,top_p,response_time,...,generated_tokens,task_id,hypothesis,dataset,model_name_or_path,generated_code,test_code,passed,result,hash
0,0,[INST] <<SYS>>\nYour goal is to write code tha...,[PYTHON]\ndef max_volume(side_sum):\n retur...,42,834,True,"['[/PYTHON]', '</s>']",0.8,0.95,1.852320,...,38,318,{'instruction': 'Your goal is to write code th...,mbpp,codellama/CodeLlama-7b-Instruct-hf,def max_volume(side_sum):\n return (side_su...,assert max_volume(8) == 18\nassert max_volume(...,False,failed:,82ffe7d05e3196418e39d9aa46bf4126eb2759a419ca25...
1,20,[INST] <<SYS>>\nYour goal is to write code tha...,[PYTHON]\ndef max_volume(sum_of_sides):\n #...,43,834,True,"['[/PYTHON]', '</s>']",0.8,0.95,5.123448,...,119,318,{'instruction': 'Your goal is to write code th...,mbpp,codellama/CodeLlama-7b-Instruct-hf,def max_volume(sum_of_sides):\n # Find the ...,assert max_volume(8) == 18\nassert max_volume(...,False,failed:,c57d9f5efe770f1ce1a429101f3bd4aee5c529e81a3a84...
2,0,[INST] <<SYS>>\nYou are tasked with writing co...,[PYTHON]\ndef max_volume(side_sum):\n retur...,42,834,True,"['[/PYTHON]', '</s>']",0.8,0.95,1.870779,...,38,318,{'instruction': 'You are tasked with writing c...,mbpp,codellama/CodeLlama-7b-Instruct-hf,def max_volume(side_sum):\n return (side_su...,assert max_volume(8) == 18\nassert max_volume(...,False,failed:,fdc20642fbdf1295bb92b21837336e90576db0996d188f...
3,20,[INST] <<SYS>>\nYou are tasked with writing co...,[PYTHON]\ndef max_volume(s):\n return (s **...,43,834,True,"['[/PYTHON]', '</s>']",0.8,0.95,2.066289,...,33,318,{'instruction': 'You are tasked with writing c...,mbpp,codellama/CodeLlama-7b-Instruct-hf,def max_volume(s):\n return (s ** 3) / 3,assert max_volume(8) == 18\nassert max_volume(...,False,failed:,e75b672fa3846b616e2f7123616d13f726edd37433f252...
4,1888,[INST] <<SYS>>\nYour goal is to write code tha...,[PYTHON]\ndef max_volume(sum_of_sides):\n #...,44,512,True,"['[/PYTHON]', '</s>']",0.8,0.95,10.906498,...,187,318,{'instruction': 'Your goal is to write code th...,mbpp,codellama/CodeLlama-7b-Instruct-hf,def max_volume(sum_of_sides):\n # Find the ...,assert max_volume(8) == 18\nassert max_volume(...,False,failed:,8b389fe9be6e07349871ef5fec081e26554631f554fa39...


In [7]:
print(mbpp_df['text'].iloc[4])

[INST] <<SYS>>
Your goal is to write code that performs the specified task.
<</SYS>>

Write a python function to find the maximum volume of a cuboid with given sum of sides.
Your code should pass these tests:

assert max_volume(8) == 18
assert max_volume(4) == 2
assert max_volume(1) == 0
Your code should start with a [PYTHON] tag and end with a [/PYTHON] tag. [/INST]


In [8]:
print(mbpp_df['generated_text'].iloc[4])

[PYTHON]
def max_volume(sum_of_sides):
    # Find the maximum volume of a cuboid with the given sum of sides
    # by using the formula for the volume of a cuboid:
    # volume = length * width * height
    # We know that the sum of the sides is equal to the length, width, and height,
    # so we can set up the following system of equations:
    # length + width + height = sum_of_sides
    # length * width * height = max_volume
    # We can solve this system of equations to find the maximum volume:
    length = sum_of_sides // 3
    width = length
    height = length
    max_volume = length * width * height
    return max_volume
[/PYTHON]


In [9]:
# TODO: from scripts.code_eval.evaluation import pass_at_k

In [10]:
prompt_list = mbpp_df["hypothesis"].unique()

In [13]:
res = []
for p in prompt_list:

    if eval(p)["instruction"] != 'You are required to write code that generates the specified output.':
        continue

    print(p)

    prompt_df = mbpp_df[mbpp_df["hypothesis"] == p]
    print("no. generations:", len(prompt_df))

    prompt_tasks = prompt_df["task_id"].unique()
    print("no. tasks:", len(prompt_tasks))

    scores = []
    for task_id in prompt_tasks:
        
        test_results = prompt_df[prompt_df["task_id"] == task_id]["passed"].tolist()
        
        if True in test_results:
            scores.append(1)
        elif False in test_results:
            scores.append(0)
        else:
            continue
        # print(scores[-1], test_results)

    scores = 1-np.array(scores)
    print(np.mean(scores))

    print("-----")

    queries = [q for q in prompt_df["text"].tolist()]
    responses = prompt_df["generated_text"].tolist()


    res.append([
        p,
        scores,
        queries,
        responses
    ])

{'instruction': 'You are required to write code that generates the specified output.'}
no. generations: 9640
no. tasks: 964
0.4170124481327801
-----
{'instruction': 'You are required to write code that generates the specified output.', 'k_shot_idxs': [2]}
no. generations: 9640
no. tasks: 964
0.4066390041493776
-----
{'instruction': 'You are required to write code that generates the specified output.', 'k_shot_idxs': [2, 7, 10]}
no. generations: 9640
no. tasks: 964
0.3900414937759336
-----


In [14]:
import pickle as pkl

filename = "../output/mbpp/mbpp_codellama_pass-10_loss_dist.pkl"
with open(filename, 'wb') as handle:
    pkl.dump(res, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [15]:
import pickle as pkl

filename = "../output/mbpp/mbpp_codellama_pass-5_loss_dist.pkl"
with open(filename, 'wb') as handle:
    pkl.dump(res, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [26]:
res[2][2][0]

'[INST] <<SYS>>\nYou are required to write code that generates the specified output.\n<</SYS>>\n\nWrite a function to find the similar elements from the given two tuple lists.\nYour code should pass these tests:\n\nassert similar_elements((3, 4, 5, 6),(5, 7, 4, 10)) == (4, 5)\nassert similar_elements((1, 2, 3, 4),(5, 4, 3, 7)) == (3, 4)\nassert similar_elements((11, 12, 14, 13),(17, 15, 14, 13)) == (13, 14)\nYour code should start with a [PYTHON] tag and end with a [/PYTHON] tag.\n[PYTHON]\ndef similar_elements(test_tup1, test_tup2):\r\n  res = tuple(set(test_tup1) & set(test_tup2))\r\n  return (res)\n[/PYTHON]\nWrite a function to find all words which are at least 4 characters long in a string by using regex.\nYour code should pass these tests:\n\nassert find_char_long(\'Please move back to stream\') == [\'Please\', \'move\', \'back\', \'stream\']\nassert find_char_long(\'Jing Eco and Tech\') == [\'Jing\', \'Tech\']\nassert find_char_long(\'Jhingai wulu road Zone 3\') == [\'Jhingai\',